# DARE-TIES

Best model merging with Dare tie

**Yaml Config**

In [ ]:
MODEL_NAME = "wasifis/Falcon-10B-merged-v2-slerp"
yaml_config = """
slices:
  - sources:
      - model: tiiuae/Falcon3-10B-Instruct
        layer_range: [0, 40]
      - model: wasifis/Falcon-10B-v1
        layer_range: [0, 40]
merge_method: slerp
base_model: tiiuae/Falcon3-10B-Instruct
parameters:
  t:
    - filter: self_attn
      value: [0, 0.5, 0.3, 0.7, 1]
    - filter: mlp
      value: [1, 0.5, 0.7, 0.3, 0]
    - value: 0.5
dtype: bfloat16
"""

In [ ]:


runtime = "GPU"

branch = "main"
trust_remote_code = True

# Install mergekit
# if branch == "main":
#     !git clone https://github.com/arcee-ai/mergekit.git
#     !cd mergekit && pip install -e . --progress-bar off
# elif branch == "mixtral":
#     !git clone -b mixtral https://github.com/arcee-ai/mergekit.git
#     !cd mergekit && pip install -qqq -e . --progress-bar off
#     !pip install -qqq -U transformers --progress-bar off

# Save config as yaml file
with open('config.yaml', 'w', encoding="utf-8") as f:
    f.write(yaml_config)

# Base CLI
if branch == "main":
    cli = "mergekit-yaml config.yaml merge --copy-tokenizer"
elif branch == "mixtral":
    cli = "mergekit-moe config.yaml merge --copy-tokenizer"

# Additional arguments
if runtime == "CPU":
    cli += " --allow-crimes --out-shard-size 1B --lazy-unpickle"
elif runtime == "GPU":
    cli += " --cuda --low-cpu-memory"
if trust_remote_code:
    cli += " --trust-remote-code"

print(cli)

# Merge models
!{cli}